In [2]:
import pandas as pd
import numpy as np
import docplex
from docplex.mp.model import Model

In [62]:
# Load the Excel file
file_path = "inst1.xlsx"
excel_data = pd.ExcelFile(file_path)

# Read data
unloading = excel_data.parse("Problem_1", index_col=None, header=0, usecols="B:E", nrows=5, skiprows=1)
cat = excel_data.parse("Problem_1", index_col=None, header=0, usecols="G:H", nrows=8, skiprows=1)
arrival = excel_data.parse("Problem_1", index_col=None, header=0, usecols="J:K", nrows=8, skiprows=1)

# indices
ships = np.arange(1, len(arrival)+1)
stations = np.arange(1, len(unloading.columns)+1)
categories = np.arange(1, len(unloading)+1)
names = {a+1: arrival.iloc[a,0] for a in range(0, len(arrival))}

# Changing input
unloading.index = categories
unloading.columns = stations
arrival = {a+1: arrival.iloc[a,1] for a in range(0, len(arrival))}
#cat = {c+1: cat.iloc[c,1] for c in range(0, len(cat))}

# BigM
BigM = 99999

# Combined parameters
conform = pd.DataFrame(index=categories, columns=stations)
for c in categories:
    for p in stations:
        conform.loc[c,p] = 1 if unloading.loc[c,p] < 1000 else 0

cat.iloc[:,0] = np.arange(1, len(cat)+1)
category = pd.DataFrame(0, index=ships, columns=categories)
for _ in range(0,len(cat)):
    category.loc[cat.iloc[_,0], cat.iloc[_,1]] = 1

In [11]:
model = Model(name="ship_sheduling", log_output=False)

In [57]:
model.clear()
X = model.integer_var_matrix(ships, stations, name="Start")
T = model.binary_var_matrix(ships, stations, name="Unloading")
E = model.binary_var_matrix(ships, ships, name="Entering")
C_max = model.integer_var(name="Makespan")

In [94]:
model.clear_constraints()

# No early unlading
model.add_constraints(X[s,1] >= arrival[s] for s in ships)

# Ship must have passed previous station to reach new station
for s in ships:
    for p in stations:
        for c in categories:
            if p > 1:
                model.add_constraint(X[s,p] >= X[s,p-1] + unloading.loc[c,p-1] * category.loc[s,c] * T[s,p-1] + 1)

# Ships can't overtake
for s1 in ships:
    for s2 in ships:
        for p in stations:
            model.add_constraint(X[s2,p] + 1 <= X[s1,p] + (1-E[s1,s2]) * BigM)

# Ships must be unloaded once
model.add_constraints(model.sum(T[s,p] for p in stations) == 1 for s in ships)

# Ships can only be unloaded at conform stations
model.add_constraints(T[s,p] * category.loc[s,c] <= conform.loc[c,p] for s in ships for p in stations for c in categories)

# Setting the makespan
model.add_constraints(C_max >= X[s,4] + unloading.loc[c,4] * category.loc[s,c] * T[s,4] for s in ships for c in categories)

[docplex.mp.LinearConstraint[](Makespan,GE,Start_1_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_1_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_1_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_1_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_1_4+7Unloading_1_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_2_4+9999Unloading_2_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_2_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_2_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_2_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_2_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_3_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_3_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_3_4+2Unloading_3_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_3_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_3_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_4_4),
 docplex.mp.LinearConstraint[](Makespan,GE,Start_4_4),
 docplex.mp.Linea

In [95]:
model.minimize(C_max)
model.solve()
print(model.solve_details)
print(f"The objective value is {model.objective_value} Periods")

status  = integer optimal solution
time    = 0.015 s.
problem = MILP
gap     = 0%

The objective value is 15.0 Periods


### Visualization

In [117]:
df = pd.DataFrame(index=np.arange(0, len(ships)*len(stations)), columns=["Task", "Start", "Finish", "Ressource"])
cnt = 0
for s in ships:
    for p in stations:
        for c in categories:
            if category.loc[s,c] == 1:
                ship_category = c
        df.iloc[cnt, 0] = names[s]
        df.iloc[cnt, 1] = (X[s,p].solution_value - 1) * 15
        if T[s,p].solution_value == 1:
            df.iloc[cnt, 2] = (X[s,p].solution_value + unloading.loc[ship_category,p]) * 15
        else:
            df.iloc[cnt, 2] = X[s,p].solution_value * 15
        df.iloc[cnt,3] = p
        cnt += 1

In [99]:
category

,1,2,3,4,5
1,0,0,0,0,1
2,1,0,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0
5,0,0,0,1,0
6,0,1,0,0,0
7,0,0,0,0,1
8,0,0,0,0,1


In [98]:
for s in ships:
    for p in stations:
        print(f"ship: {s}, station: {p}, unloadiong: {T[s,p]. solution_value}")

ship: 1, station: 1, unloadiong: 0
ship: 1, station: 2, unloadiong: 0
ship: 1, station: 3, unloadiong: 0
ship: 1, station: 4, unloadiong: 1.0
ship: 2, station: 1, unloadiong: 0
ship: 2, station: 2, unloadiong: 0
ship: 2, station: 3, unloadiong: 1.0
ship: 2, station: 4, unloadiong: 0
ship: 3, station: 1, unloadiong: 0
ship: 3, station: 2, unloadiong: 0
ship: 3, station: 3, unloadiong: 0
ship: 3, station: 4, unloadiong: 1.0
ship: 4, station: 1, unloadiong: 0
ship: 4, station: 2, unloadiong: 0
ship: 4, station: 3, unloadiong: 0
ship: 4, station: 4, unloadiong: 1.0
ship: 5, station: 1, unloadiong: 0
ship: 5, station: 2, unloadiong: 0
ship: 5, station: 3, unloadiong: 0
ship: 5, station: 4, unloadiong: 1.0
ship: 6, station: 1, unloadiong: 0
ship: 6, station: 2, unloadiong: 1.0
ship: 6, station: 3, unloadiong: 0
ship: 6, station: 4, unloadiong: 0
ship: 7, station: 1, unloadiong: 0
ship: 7, station: 2, unloadiong: 0
ship: 7, station: 3, unloadiong: 0
ship: 7, station: 4, unloadiong: 1.0
ship: 

In [118]:
df

,Task,Start,Finish,Ressource
0,Exxon,0.0,15.0,1
1,Exxon,15.0,30.0,2
2,Exxon,30.0,45.0,3
3,Exxon,45.0,165.0,4
4,Nina,0.0,15.0,1
5,Nina,15.0,30.0,2
6,Nina,30.0,150.0,3
7,Nina,150.0,165.0,4
8,Moers,60.0,75.0,1
9,Moers,75.0,90.0,2


In [119]:
import matplotlib.pyplot as plt
import plotly.figure_factory as ff

colors = {1: "rgb(255, 0, 0)",
         2: "rgb(0, 255, 0)",
         3: "rgb(0, 0, 255)",
         4: "rgb(150, 150, 150)"}

fig = ff.create_gantt(df, colors=colors, index_col="Ressource", show_colorbar=True, group_tasks=True, bar_width=0.4)
fig.update_layout(xaxis_type='linear', autosize=False, width=800, height=600)
fig.show()

In [90]:
arrival

{1: 1, 2: 1, 3: 5, 4: 4, 5: 1, 6: 6, 7: 3, 8: 7}